In [197]:
import requests
import os
import json
import plotly
import plotly.plotly as py
import plotly.graph_objs as go
from plotly import tools
import numpy as np
from shapely.geometry import Polygon
from shapely.geometry import Point
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
plotly.offline.init_notebook_mode(connected=True)
# source: https://plot.ly/python/horizontal-bar-charts/

In [237]:
# Database Connection Configuration
BASE_URL = 'http://172.26.38.57:5984'
USERNAME = 'admin'
PASSWORD = 'password'
db = requests.Session()
db.auth = (USERNAME, PASSWORD)

<h3> Sins By Time Range </h3>

In [252]:
url = BASE_URL + '/analysis_extended/_design/summary/_view/count_by_time_range?group=true'
response = db.get(url)


In [253]:
data = response.json()

In [254]:
top_labels = ['sloth','greed','gluttony','wrath','pride','lust','envy']

In [255]:
x_data = []
y_data = []
for i in range(len(data['rows'])):
    x_data.append(data['rows'][i]['value'])
    y_data.append(data['rows'][i]['key'])

In [257]:
y_data

['1.Early morning',
 '2.Morning',
 '3.Late Morning',
 '4.Early Afternoon',
 '5.Late Afternoon',
 '6.Early Evening',
 '7.Evening',
 '8.Night']

In [258]:
for i in range(len(x_data)):
    total = sum(x_data[i])
    for j in range(len(x_data[i])):
        x_data[i][j] = round((x_data[i][j] / total) * 100,1)
        

In [259]:
colors = ['rgba(38, 24, 74, 0.8)', 'rgba(71, 58, 131, 0.8)',
          'rgba(122, 120, 168, 0.8)', 'rgba(164, 163, 204, 0.85)',
          'rgba(190, 192, 213, 1)','rgba(190, 192, 213, 1)','rgba(190, 192, 213, 1)']

traces = []

for i in range(0, len(x_data[0])):
    for xd, yd in zip(x_data, y_data):
        
        traces.append(go.Bar(
            x=[xd[i]],
            y=[yd],
            orientation='h',
            marker=dict(
                color=colors[i],
                line=dict(
                        color='rgb(248, 248, 249)',
                        width=1)
            )
        ))

layout = go.Layout(
    xaxis=dict(
        showgrid=False,
        showline=False,
        showticklabels=False,
        zeroline=False,
        domain=[0.15, 1]
    ),
    yaxis=dict(
        showgrid=False,
        showline=False,
        showticklabels=False,
        zeroline=False,
    ),
    barmode='stack',
    paper_bgcolor='rgb(248, 248, 255)',
    plot_bgcolor='rgb(248, 248, 255)',
    margin=dict(
        l=120,
        r=10,
        t=140,
        b=80
    ),
    showlegend=False,
)

annotations = []

for yd, xd in zip(y_data, x_data):
    # labeling the y-axis
    annotations.append(dict(xref='paper', yref='y',
                            x=0.14, y=yd,
                            xanchor='right',
                            text=str(yd),
                            font=dict(family='Arial', size=14,
                                      color='rgb(67, 67, 67)'),
                            showarrow=False, align='right'))
    # labeling the first percentage of each bar (x_axis)
    annotations.append(dict(xref='x', yref='y',
                            x=xd[0] / 2, y=yd,
                            text=str(xd[0]) + '%',
                            font=dict(family='Arial', size=14,
                                      color='rgb(248, 248, 255)'),
                            showarrow=False))
    # labeling the first Likert scale (on the top)
    if yd == y_data[-1]:
        annotations.append(dict(xref='x', yref='paper',
                                x=xd[0] / 2, y=1.1,
                                text=top_labels[0],
                                font=dict(family='Arial', size=14,
                                          color='rgb(67, 67, 67)'),
                                showarrow=False))
    space = xd[0]
    for i in range(1, len(xd)):
            # labeling the rest of percentages for each bar (x_axis)
            annotations.append(dict(xref='x', yref='y',
                                    x=space + (xd[i]/2), y=yd, 
                                    text=str(xd[i]) + '%',
                                    font=dict(family='Arial', size=14,
                                              color='rgb(248, 248, 255)'),
                                    showarrow=False))
            # labeling the Likert scale
            if yd == y_data[-1]:
                annotations.append(dict(xref='x', yref='paper',
                                        x=space + (xd[i]/2), y=1.1,
                                        text=top_labels[i],
                                        font=dict(family='Arial', size=14,
                                                  color='rgb(67, 67, 67)'),
                                        showarrow=False))
            space += xd[i]

layout['annotations'] = annotations

fig = go.Figure(data=traces, layout=layout)
#py.iplot(fig, filename='bar-colorscale')
plotly.offline.plot(fig, filename='sins_by_timerange')

/Users/indah/anaconda3/lib/python3.7/site-packages/plotly/offline/offline.py:524: UserWarning:

Your filename `sins_by_timerange` didn't end with .html. Adding .html to the end of your file.



'sins_by_timerange.html'

<h3> Most Common Travel Destinations </h3>

In [247]:
url2 = BASE_URL + '/analysis_extended/_design/summary/_view/travel_dest?group=true'
response2 = db.get(url2)

In [248]:
data2 = response2.json()

In [249]:
key_data = []
count_data = []
min_data = []
for i in range(len(data2['rows'])):
    key_data.append(data2['rows'][i]['key'])
    count_data.append(data2['rows'][i]['value']['count'])
    min_data.append(round(data2['rows'][i]['value']['min'],1))

In [250]:
l1, l2, l3 = (list(t) for t in zip(*sorted(zip(count_data, min_data, key_data))))

In [251]:

y_saving = l1[-10:]
y_net_worth = l2[-10:]
x_saving = l3[-10:]
x_net_worth = l3[-10:]

trace0 = go.Bar(
    x=y_saving,
    y=x_saving,
    marker=dict(
        color='rgba(50, 171, 96, 0.6)',
        line=dict(
            color='rgba(50, 171, 96, 1.0)',
            width=1),
    ),
    name='Number of Tweets By Users Who Travel on The Same Day',
    orientation='h',
)
trace1 = go.Scatter(
    x=y_net_worth,
    y=x_net_worth,
    mode='lines+markers',
    line=dict(
        color='rgb(128, 0, 128)'),
    name='Minimum Distance From Victoria (in KM)',
)
layout = dict(
    title='Most Common Travel Destinations of Victorians',
    yaxis=dict(
        showgrid=False,
        showline=False,
        showticklabels=True,
        domain=[0, 0.85],
    ),
    yaxis2=dict(
        showgrid=False,
        showline=True,
        showticklabels=False,
        linecolor='rgba(102, 102, 102, 0.8)',
        linewidth=2,
        domain=[0, 0.85],
    ),
    xaxis=dict(
        zeroline=False,
        showline=False,
        showticklabels=True,
        showgrid=True,
        domain=[0, 0.42],
    ),
    xaxis2=dict(
        zeroline=False,
        showline=False,
        showticklabels=True,
        showgrid=True,
        domain=[0.47, 1],
        side='top',
        dtick=5000,
    ),
    legend=dict(
        x=0.029,
        y=1.038,
        font=dict(
            size=10,
        ),
    ),
    margin=dict(
        l=100,
        r=20,
        t=70,
        b=70,
    ),
    paper_bgcolor='rgb(248, 248, 255)',
    plot_bgcolor='rgb(248, 248, 255)',
)

annotations = []

y_s = np.round(y_saving, decimals=2)
y_nw = np.rint(y_net_worth)

# Adding labels
for ydn, yd, xd in zip(y_nw, y_s, x_saving):
    # labeling the scatter savings
    annotations.append(dict(xref='x2', yref='y2',
                            y=xd, x=ydn - 2000,
                            text='{:,}'.format(ydn),
                            font=dict(family='Arial', size=12,
                                      color='rgb(128, 0, 128)'),
                            showarrow=False))
    # labeling the bar net worth
    annotations.append(dict(xref='x1', yref='y1',
                            y=xd, x=yd + 3,
                            text=str(yd),
                            font=dict(family='Arial', size=12,
                                      color='rgb(50, 171, 96)'),
                            showarrow=False))
# Source
annotations.append(dict(xref='paper', yref='paper',
                        x=-0.05, y=-0.109,
                        text='Aggregated Based on Multiple Tweets of The Same User on The Same Day',
                        font=dict(family='Arial', size=10,
                                  color='rgb(150,150,150)'),
                        showarrow=False))

layout['annotations'] = annotations

# Creating two subplots
fig = tools.make_subplots(rows=1, cols=2, specs=[[{}, {}]], shared_xaxes=True,
                          shared_yaxes=False, vertical_spacing=0.001)

fig.append_trace(trace0, 1, 1)
fig.append_trace(trace1, 1, 2)

fig['layout'].update(layout)
plotly.offline.plot(fig, filename='travel_dest')

This is the format of your plot grid:
[ (1,1) x1,y1 ]  [ (1,2) x2,y2 ]



/Users/indah/anaconda3/lib/python3.7/site-packages/plotly/offline/offline.py:524: UserWarning:

Your filename `travel_dest` didn't end with .html. Adding .html to the end of your file.



'travel_dest.html'

<h3> City Happiness </h3>

In [321]:
url3 = BASE_URL + '/analysis_extended/_design/summary/_view/travel_vic?group=true'
response3 = db.get(url3)

In [322]:
data3 = response3.json()

In [323]:
key_data = []
count_data = []
mean_data = []
for i in range(len(data3['rows'])):
    key_data.append(data3['rows'][i]['key'])
    count = data3['rows'][i]['value']['count']
    count_data.append(count)
    sum_pol = data3['rows'][i]['value']['sum']
    avg_polarity = sum_pol/count
    mean_data.append(avg_polarity)

In [324]:
l1, l2, l3 = (list(t) for t in zip(*sorted(zip(count_data, mean_data, key_data))))

In [326]:

y_saving = l1[-10:]
y_net_worth = l2[-10:]
x_saving = l3[-10:]
x_net_worth = l3[-10:]

trace0 = go.Bar(
    x=y_saving,
    y=x_saving,
    marker=dict(
        color='rgba(50, 171, 96, 0.6)',
        line=dict(
            color='rgba(50, 171, 96, 1.0)',
            width=1),
    ),
    name='Count of Tweets',
    orientation='h',
)
trace1 = go.Scatter(
    x=y_net_worth,
    y=x_net_worth,
    mode='lines+markers',
    line=dict(
        color='rgb(128, 0, 128)'),
    name='Sentiment Polarity [-1,1]',
)
layout = dict(
    title='Victoria Cities Based on Count of Tweets and Average Sentiment Polarity',
    yaxis=dict(
        showgrid=False,
        showline=False,
        showticklabels=True,
        domain=[0, 0.85],
    ),
    yaxis2=dict(
        showgrid=False,
        showline=True,
        showticklabels=False,
        linecolor='rgba(102, 102, 102, 0.8)',
        linewidth=2,
        domain=[0, 0.85],
    ),
    xaxis=dict(
        zeroline=False,
        showline=False,
        showticklabels=True,
        showgrid=True,
        domain=[0, 0.42],
    ),
    xaxis2=dict(
        zeroline=False,
        showline=False,
        showticklabels=True,
        showgrid=True,
        domain=[0.47, 1],
        side='top'
    ),
    legend=dict(
        x=0.029,
        y=1.038,
        font=dict(
            size=10,
        ),
    ),
    margin=dict(
        l=100,
        r=20,
        t=70,
        b=70,
    ),
    paper_bgcolor='rgb(248, 248, 255)',
    plot_bgcolor='rgb(248, 248, 255)',
)

annotations = []

y_s = np.round(y_saving, decimals=2)
y_nw = np.rint(y_net_worth)

# Adding labels
for ydn, yd, xd in zip(y_nw, y_s, x_saving):
    # labeling the scatter savings
    annotations.append(dict(xref='x2', yref='y2',
                            y=xd, x=ydn - 0.1,
                            text='{:,}'.format(ydn),
                            font=dict(family='Arial', size=12,
                                      color='rgb(128, 0, 128)'),
                            showarrow=False))
    # labeling the bar net worth
    annotations.append(dict(xref='x1', yref='y1',
                            y=xd, x=yd + 0.1,
                            text=str(yd),
                            font=dict(family='Arial', size=12,
                                      color='rgb(50, 171, 96)'),
                            showarrow=False))
# Source
annotations.append(dict(xref='paper', yref='paper',
                        x=-0.05, y=-0.109,
                        text='',
                        font=dict(family='Arial', size=10,
                                  color='rgb(150,150,150)'),
                        showarrow=False))

layout['annotations'] = annotations

# Creating two subplots
fig = tools.make_subplots(rows=1, cols=2, specs=[[{}, {}]], shared_xaxes=True,
                          shared_yaxes=False, vertical_spacing=0.001)

fig.append_trace(trace0, 1, 1)
fig.append_trace(trace1, 1, 2)

fig['layout'].update(layout)
plotly.offline.plot(fig, filename='happy_vic')

This is the format of your plot grid:
[ (1,1) x1,y1 ]  [ (1,2) x2,y2 ]



/Users/indah/anaconda3/lib/python3.7/site-packages/plotly/offline/offline.py:524: UserWarning:

Your filename `happy_vic` didn't end with .html. Adding .html to the end of your file.



'happy_vic.html'

<h3> City Street </h3>

In [297]:
url4 = BASE_URL + '/analysis_street/_design/summary/_view/view?group=true'
response4 = db.get(url4)

In [298]:
data4 = response4.json()

In [310]:
key_data = []
count_data = []
mean_data = []
for i in range(len(data4['rows'])):
    key_data.append(data4['rows'][i]['key'])
    count = data4['rows'][i]['value']['count']
    count_data.append(count)
    sum_pol = data4['rows'][i]['value']['sum']
    avg_polarity = sum_pol/count
    mean_data.append(avg_polarity)

In [311]:
l1, l2, l3 = (list(t) for t in zip(*sorted(zip(count_data, mean_data, key_data))))

In [319]:

y_saving = l1[-10:]
y_net_worth = l2[-10:]
x_saving = l3[-10:]
x_net_worth = l3[-10:]

trace0 = go.Bar(
    x=y_saving,
    y=x_saving,
    marker=dict(
        color='rgba(50, 171, 96, 0.6)',
        line=dict(
            color='rgba(50, 171, 96, 1.0)',
            width=1),
    ),
    name='Count of Tweets with the closest distance to the street',
    orientation='h',
)
trace1 = go.Scatter(
    x=y_net_worth,
    y=x_net_worth,
    mode='lines+markers',
    line=dict(
        color='rgb(128, 0, 128)'),
    name='Average Closest Distance From All Cities in Victoria (in KM)',
)
layout = dict(
    title='Most Common Closest Streets',
    yaxis=dict(
        showgrid=False,
        showline=False,
        showticklabels=True,
        domain=[0, 0.85],
    ),
    yaxis2=dict(
        showgrid=False,
        showline=True,
        showticklabels=False,
        linecolor='rgba(102, 102, 102, 0.8)',
        linewidth=2,
        domain=[0, 0.85],
    ),
    xaxis=dict(
        zeroline=False,
        showline=False,
        showticklabels=True,
        showgrid=True,
        domain=[0, 0.42],
    ),
    xaxis2=dict(
        zeroline=False,
        showline=False,
        showticklabels=True,
        showgrid=True,
        domain=[0.47, 1],
        side='top'
    ),
    legend=dict(
        x=0.029,
        y=1.038,
        font=dict(
            size=10,
        ),
    ),
    margin=dict(
        l=100,
        r=20,
        t=70,
        b=70,
    ),
    paper_bgcolor='rgb(248, 248, 255)',
    plot_bgcolor='rgb(248, 248, 255)',
)

annotations = []

y_s = np.round(y_saving, decimals=2)
y_nw = np.rint(y_net_worth)

# Adding labels
for ydn, yd, xd in zip(y_nw, y_s, x_saving):
    # labeling the scatter savings
    annotations.append(dict(xref='x2', yref='y2',
                            y=xd, x=ydn - 10,
                            text='{:,}'.format(ydn),
                            font=dict(family='Arial', size=12,
                                      color='rgb(128, 0, 128)'),
                            showarrow=False))
    # labeling the bar net worth
    annotations.append(dict(xref='x1', yref='y1',
                            y=xd, x=yd + 3,
                            text=str(yd),
                            font=dict(family='Arial', size=12,
                                      color='rgb(50, 171, 96)'),
                            showarrow=False))
# Source
annotations.append(dict(xref='paper', yref='paper',
                        x=-0.05, y=-0.109,
                        text='',
                        font=dict(family='Arial', size=10,
                                  color='rgb(150,150,150)'),
                        showarrow=False))

layout['annotations'] = annotations

# Creating two subplots
fig = tools.make_subplots(rows=1, cols=2, specs=[[{}, {}]], shared_xaxes=True,
                          shared_yaxes=False, vertical_spacing=0.001)

fig.append_trace(trace0, 1, 1)
fig.append_trace(trace1, 1, 2)

fig['layout'].update(layout)
plotly.offline.plot(fig, filename='closest_street')

This is the format of your plot grid:
[ (1,1) x1,y1 ]  [ (1,2) x2,y2 ]



/Users/indah/anaconda3/lib/python3.7/site-packages/plotly/offline/offline.py:524: UserWarning:

Your filename `closest_street` didn't end with .html. Adding .html to the end of your file.



'closest_street.html'